In [1]:
import pandas as pd
import numpy as np
import csv
import os

In [2]:
r = csv.reader(open('project.csv')) 
lines = list(r)


In [3]:
# This function make change csv file, user type coordinate of elements which want to change
def new_vol(address, new_value):
    filepath='project.csv'
    dialect_params = dict(delimiter=',')
    col_num, row_num = address
    with open(filepath, "r") as csvfile:
        reader = csv.reader(csvfile, **dialect_params)
        lines = []

        for current_line in reader:
            if reader.line_num == row_num:
                current_line[col_num-1] = new_value
            lines.append(current_line)

        csvfile.seek(0)
        writer = csv.writer(open('outo.csv', 'w',newline='')) 
        writer.writerows(lines) 


new_vol(address=(1, 5), new_value="Good!")

In [4]:
df=pd.read_csv('outo.csv')
df

,d,f
0,1,1
1,1,1
2,2,3
3,Good!,4
4,5,5
5,8,6
6,13,7
7,21,8
8,34,9
9,55,1


In [5]:
#this function cut our file to different files, after that we will cipher them and distribute, user must choose how many rows will be in every file
def split(filehandler, delimiter=',', row_limit=10,
          output_name_template='cut_files_%s.csv', output_path='.', keep_headers=True):
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w',newline=''), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w',newline=''), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)
        
split(open('outo.csv', 'r'), row_limit=4);

In [6]:
#example of cut file
cut=pd.read_csv('cut_files_1.csv')
cut

,d,f
0,1,1
1,1,1
2,2,3
3,Good!,4


### Shamir Secret share

In [7]:
#This functions use implementation of Shamir's secret scheme to generate and share keys
from shamir import make_random_shares
from shamir import recover_secret

def key_gen (theshold, members) :
    return make_random_shares(theshold, members)

def key_recover (shares) :
    return recover_secret(shares)


### AES file cipher

In [8]:
#implementation of AES encrypt/decrypt file in Cipher Block Chaining mode (16 bit) 

from hashlib import md5
from base64 import b64decode
from base64 import b64encode
from Crypto import Random
from Crypto.Cipher import AES


# Padding for the input string --not
# related to encryption itself.
BLOCK_SIZE = 16  # Bytes
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE).encode()
unpad = lambda s: s[:-ord(s[len(s) - 1:])]


class AESCipher:
    
    def __init__(self, key):
        self.key = key.to_bytes(16, 'big')

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return (iv + cipher.encrypt(raw))
    
    def decrypt(self, enc):
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[16:]))
    
    def file_encrypt(self, name_in, name_out):
        fin = open(name_in, "rb")
        fout = open(name_out, "wb")        
        e = self.encrypt(fin.read())
        fout.write(e)
        
        fin.close()
        fout.close()
        
    def file_decrypt(self, name_in, name_out):
        fin = open(name_in, "rb")
        fout = open(name_out, "wb")        
        
        d = fin.read()
        fout.write(self.decrypt(d))
        
        fin.close()
        fout.close()
    

chipher = AESCipher(12).encrypt(b'halo')
AESCipher(12).decrypt(chipher)

b'halo'

In [9]:
key, shares = key_gen(3, 6)
AESCipher(key).file_encrypt("cut_files_1.csv", "encrypted.csv")
AESCipher(key).file_decrypt("encrypted.csv", "decrypted.csv")
